# Notebook to summarize the tag data

### This notebook aims to provide you with simple functions for overviewing your biologging data

**As such, we expect you to have pre-processed the raw data of your biologging campaign as previously introduced.**

# 0. Notebook Initialization

In [ ]:
# needed for now...
!pip install rich zstandard
!pip install "xarray-healpy @ git+https://github.com/iaocea/xarray-healpy.git@0ffca6058f4008f4f22f076e2d60787fcf32ac82"
!pip install movingpandas more_itertools
!pip install xarray --upgrade
!pip install xdggs
!pip install git+https://github.com/IAOCEA/healpix-convolution

In [ ]:
# Python imports
import os
import sys
import warnings
from pathlib import Path

import holoviews as hv
import hvplot
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import s3fs
import tqdm
import xarray as xr

**Update the variables below with your data.**

In [ ]:
TAG_ROOT = "path_to_your_formatted_tag_folder/"

In [ ]:
storage_options = {
    "anon": False,
    "profile": "gfts",
    "client_kwargs": {
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net/",
        "region_name": "gra",
    },
}
s3 = s3fs.S3FileSystem(**storage_options)
tag_names = [
    tn.replace(TAG_ROOT, "") for tn in s3.ls(TAG_ROOT) if not tn.endswith(".csv")
]
print(f'Found {len(tag_names)} in tag folder(s) in: "{TAG_ROOT}."')

# 1. Release Locations Overview

In this first section, we visualize the release locations. 
We have implemented static and dynamic visualizations (which can be useful if you want to interact with the map

In [ ]:
def aggregate_release_locations(
    paths: list[str], labels: list[str], storage_options: dict
):
    if len(paths) != len(labels):
        raise ValueError(
            "the number of labels provided is different than the number of paths."
        )

    pbar = tqdm.tqdm(paths, file=sys.stdout)
    lons, lats = [], []
    aws_prefix = "s3://"
    for p in pbar:
        if not p.startswith(aws_prefix):
            p = aws_prefix + p
        df = pd.read_csv(
            p, storage_options=storage_options, index_col="event_name"
        )  # .assign(time=lambda df: pd.to_datetime(df["time"]))
        s = df.loc["release"]
        lon = s.longitude
        lat = s.latitude
        lons.append(lon)
        lats.append(lat)
    data = {"tag_name": labels, "longitude": lons, "latitude": lats}
    pbar.close()
    return pd.DataFrame.from_dict(data, orient="columns")

In [ ]:
df_paths = s3.glob(TAG_ROOT + "*/tagging_events.csv")
df = aggregate_release_locations(df_paths, tag_names, storage_options)
df.head(3)

## a. Static Visualization

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import pandas as pd


def static_plot(df: pd.DataFrame, margin=2.5, label_col: str = None):
    """Static Visualization of data points.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe of the data points
    margin : float, default: 2.5
        Offset to extend the longitude and latitude ranges of the plot
    label_col : str, default: None
        Name of the column to label the points. If set to None, no labels are added

    Returns
    -------
    figure : matplotlib.figure.Figure
        The figure
    """

    ccr = ccrs.PlateCarree()

    fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={"projection": ccr})

    gridlines_kwargs = {
        "crs": ccr,
        "draw_labels": True,
        "linewidth": 0.6,
        "color": "gray",
        "alpha": 0.5,
        "linestyle": "-.",
    }
    # coastlines
    ax.add_feature(cf.COASTLINE.with_scale("10m"), lw=0.5)
    # gridlines
    gl = ax.gridlines(**gridlines_kwargs)
    # gl.right_labels = False
    # gl.top_labels = False

    ax.scatter(df["longitude"], df["latitude"], s=20)

    if label_col is not None:
        for i, row in df.iterrows():
            ax.text(
                row["longitude"],
                row["latitude"],
                row[label_col],
                fontsize=10,
                ha="right",
                transform=ccr,
            )

    ax.set_title("")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_extent(
        [
            df["longitude"].min() - margin,
            df["longitude"].max() + margin,
            df["latitude"].min() - margin,
            df["latitude"].max() + margin
        ], crs=ccr)

    return fig

In [ ]:
figure = static_plot(df, label_col=None)

In [ ]:
figure.savefig("release_locations.png")

## b. Dynamic Visualization

In [ ]:
import hvplot.pandas
import pandas as pd
import geoviews as gv
import cartopy.crs as ccrs

def dynamic_plot(df: pd.DataFrame, margin=2.5, **points_kwargs):
    """Plots data points.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe of the data points
    margin : float, default: 2.5
        Offset to extend the longitude and latitude ranges of the plot

    Returns
    -------
    plot : holoviews.Overlay
        The combined plot of the points with coastlines and borders

    Other Parameters
    ----------------
    **points_kwargs : dict
        Additional keyword arguments to pass to ``df.hvplot.points``
    """

    points = df.hvplot.points(
        x="longitude", y="latitude", geo=True,
        projection=ccrs.PlateCarree(),
        hover_cols=["tag_name"],
        xlim=[df["longitude"].min() - margin, df["longitude"].max() + margin],
        ylim=[df["latitude"].min() - margin, df["latitude"].max() + margin],
        **points_kwargs
    )
    coastlines = gv.feature.coastline()
    return (coastlines * points)

In [ ]:
plot = dynamic_plot(df, size=20, title="Release Locations")
plot

# 2. Summary Table

In this second section, we create a table (using `pandas`) that summarises the following information for each tag:
- The name/id of the tag, defined by the name of the tag folder.
- Whether it has acoustic detections.
- Whether it has a final position.
- Start and end times.
- Duration in days.

**NB: Note that the section relies (and thus depends) on the `pangeo-fish` package:**

In [ ]:
import sys

sys.path.append("path_to_pangeo_fish")

# or %pip install pangeo-fish

In [ ]:
from pangeo_fish.io import open_tag

In [ ]:
import numpy as np
import xarray as xr

def summarize_tag(tag: xr.DataTree):
    """Summarizes a tag.

    Parameters
    ----------
    tag : xarray.DataTree
        The tag

    Returns
    -------
    has_acoustic_detection : bool
        Whether `tag` has acoustic detections
    has_final_position : bool
        Whether `tag` has a final location
    start, end : numpy.datetime64
        Start and end times of `tag`
    duration : int
        Duration of `tag` in days

    See Also
    --------
    pangeo_fish.io.open_tag : To load a tag.
    """
    #TODO: improve acoustic boolean assignment
    has_acoustic_detection = len(tag.groups) == 5
    has_final_position = (
        bool(
            np.isnan(tag.tagging_events.isel(event_name=1)["longitude"].to_numpy())
            or np.isnan(tag.tagging_events.isel(event_name=1)["latitude"].to_numpy())
        )
        == False
    )
    start = np.datetime64(tag["dst"].isel(time=0).time.to_numpy())
    end = np.datetime64(tag["dst"].isel(time=-1).time.to_numpy())
    delta = end - start
    duration_in_days = int(delta / np.timedelta64(1, "D"))
    return has_acoustic_detection, has_final_position, start, end, duration_in_days

In [ ]:
pbar = tqdm.tqdm(tag_names, file=sys.stdout)
summary = {}
failed_tags = []
tag_root = TAG_ROOT if TAG_ROOT.startswith("s3://") else "s3://" + TAG_ROOT
for tag_name in pbar:
    try:
        summary[tag_name] = summarize_tag(open_tag(tag_root, tag_name, storage_options))
    except Exception as e:
        print(f"EXCEPTION {tag_name}: {str(e)}.")
        failed_tags.append(tag_name)

pbar.close()

# summary = {}
# for tag_name in tag_dirs:
#     print(tag_name)
#     summary[tag_name] = summarize_tag(
#         open_tag(tag_root, tag_name, storage_options)
#     )

In [ ]:
import pandas as pd

columns = [
    "Acoustic Detection",
    "Has Final Position",
    "Start",
    "End",
    "Duration [days]",
]
df = pd.DataFrame.from_dict(summary, orient="index", columns=columns)
df.head(3)

you can sort `df` with the tags' duration, names etc.

In [ ]:
df.sort_values(by=[columns[-1]], inplace=True) # by durations

In [ ]:
output_folder = Path(".")
table_fn = "csv_filename.csv"

In [ ]:
path = Path(output_folder)
path.mkdir(parents=True, exist_ok=True)
df.to_csv(path / table_fn, index_label="Tag Name")